In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout,Conv2D,MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

print('done importing')

In [2]:
IMAGE_SIZE = (256, 256)
path = '../input/gan-art-vs-human-art/data'

datagen = ImageDataGenerator(rescale = 1/255)
dataset = datagen.flow_from_directory(path,
                                      target_size = IMAGE_SIZE,
                                      color_mode = 'rgb',
                                      batch_size = 32,
                                      class_mode = 'binary',
                                      shuffle = True,
                                      seed = 52)

print('done working with data')

In [3]:
x, y = next(dataset)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.2)

print('splitting done')

In [5]:
model = Sequential()

#layer 1
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape = (256, 256, 3)))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

#layer 2
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

#layer 3
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

#layer 4
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(2, 2))
model.add(BatchNormalization())

#flatten for density layers
model.add(Flatten())

#layer 5
model.add(Dense(128, activation = 'relu'))
model.add(Dense(256, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'softmax'))

#sum
model.summary()

In [6]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#stopping overfit
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor = 'val_loss', mode = 'min',verbose = 1, patience = 5)

print('done for overfitting')

In [7]:
#start
history = model.fit(x_train,
                    y_train,
                    validation_data = (x_val, y_val),
                    epochs = 30,
                    callbacks = [early_stop],
                    shuffle = True)

In [8]:
#loss graph
plt.plot(history.history['loss'],label='train loss')
plt.plot(history.history['val_loss'],label='val loss')
plt.legend()

In [9]:
#save the loss graph
plt.savefig('loss-graph.png')
plt.show()

In [10]:
# accuracies
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()

In [11]:
#save the acc graph
plt.savefig('acc-graph.png')
plt.show()